In [ ]:
#import beautiful soup
import sys
!{sys.executable} -m pip install bs4

In [ ]:
#Jalankan source code web scrap
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
from datetime import date
import datetime
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import statsmodels.stats.api
import statsmodels.formula.api

today = date.today()
today=date.strftime(today,'%d/%m/%Y')
urlheader = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:91.0) Gecko/20100101 Firefox/91.0",
    "X-Requested-With": "XMLHttpRequest"
}
filename=['E:\\BTC_USD Bitfinex Historical Data.csv'] #Pastikan sesuai dengan tempat Directory penyimpanan file yang sudah kalian download
url = "https://www.investing.com/instruments/HistoricalDataAjax"


for each in filename:
    with open(each, 'r') as readFile:
        reader = csv.reader(readFile)
        lines = list(reader)
    startdate=lines[1][0]
    startdate=datetime.datetime.strptime(startdate,'%d-%b-%y')
    startdate += datetime.timedelta(days=1)
    startdate=startdate.strftime('%d/%m/%Y')
    readFile.close()
    if (each == 'E:\\BTC_USD Bitfinex Historical Data.csv'): #Pastikan sesuai dengan tempat Directory penyimpanan file yang sudah kalian download 
        header="BTC/USD Bitfinex Historical Data"
        curr_id="945629"
        smlID= "145284"
        column_name='BTC Price'
 
    payload = {'header': header, 
               'st_date': startdate, 'end_date': today, 
               'sort_col': 'date', 'action': 'historical_data', 
               'smlID': smlID, 'sort_ord': 'DESC', 'interval_sec': 'Daily', 'curr_id': curr_id}

    req = requests.post(url, headers=urlheader, data=payload)
    soup = BeautifulSoup(req.content, "lxml")
    table = soup.find('table', id="curr_table")
    split_rows = table.find_all("tr")
    rows = table.findAll('tr')
    header_text = []
    headers = rows[0]
    
    for th in headers.findAll('th'):
        header_text.append(th.text)
        
    del header_text[2:]
    
    row_text_array = []
    for row in rows[1:]:
        row_text = []
        for row_element in row.findAll(['th', 'td']):
            row_text.append(row_element.text.replace('\n', '').strip())
        del row_text[2:]
        row_text_array.append(row_text)
    
    df = pd.read_csv(each)
    
    df1 = pd.DataFrame(row_text_array)
    
    df1.rename(columns={0:'Date',1:column_name}, inplace=True)
    df=df.append(df1,sort=False)
    df=df.dropna()
    df['Date']= pd.to_datetime(df['Date'])
    df = df.sort_values('Date',ascending=False)
    
    df['Date']= df['Date'].dt.strftime('%d-%b-%y')
    print(df.head(3))
    df.to_csv(each,index=False)

        Date     Price      Open      High       Low   Vol. Change %
0  12-Oct-21  57,561.0  57,468.0  57,633.0  56,443.0  6.46K    0.16%
1  11-Oct-21  57,468.0  54,701.0  57,802.5  54,430.0  7.51K    5.06%
2  10-Oct-21  54,701.0  54,978.0  56,414.0  54,173.0  5.84K   -0.50%


In [ ]:
#Menampilkan n data sementara
df.head(4)

,Date,Price,Open,High,Low,Vol.,Change %
0,12-Oct-21,"57,561.0","57,468.0","57,633.0","56,443.0",6.46K,0.16%
1,11-Oct-21,"57,468.0","54,701.0","57,802.5","54,430.0",7.51K,5.06%
2,10-Oct-21,"54,701.0","54,978.0","56,414.0","54,173.0",5.84K,-0.50%
3,09-Oct-21,"54,978.0","53,925.0","55,443.7","53,711.0",2.44K,1.95%


In [ ]:
#Menampilkan deksripsi data frame sementara
df.describe()

,Date,Price,Open,High,Low,Vol.,Change %
count,3537,3537,3537,3537,3537,3537,3537
unique,3537,3055,3044,3003,3019,1922,1323
top,31-Aug-17,5.1,5.1,5.1,5.1,-,0.00%
freq,1,26,26,26,26,660,135


In [ ]:
#Menampilkan dimensi data frame sementar
df.shape

(3537, 7)

In [ ]:
#Menampilkan tipe data kolom data frame sementara
df.dtypes

Date        object
Price       object
Open        object
High        object
Low         object
Vol.        object
Change %    object
dtype: object

In [ ]:
#Menampilkan informasi kolom data frame sementara
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3537 entries, 0 to 3536
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date      3537 non-null   object
 1   Price     3537 non-null   object
 2   Open      3537 non-null   object
 3   High      3537 non-null   object
 4   Low       3537 non-null   object
 5   Vol.      3537 non-null   object
 6   Change %  3537 non-null   object
dtypes: object(7)
memory usage: 221.1+ KB


In [ ]:
#Ubah tipe data kolom yang diperlukan
df['Date'] = pd.to_datetime(df['Date'])
df['Price'] = df['Price'].str.replace(',', '').astype(float)
df['Open'] = df['Open'].str.replace(',', '').astype(float)
df['High'] = df['High'].str.replace(',', '').astype(float)
df['Low'] = df['Low'].str.replace(',', '').astype(float)

In [ ]:
#Menampilkan n data sebenarnya
df.head(4)

,Date,Price,Open,High,Low,Vol.,Change %
0,2021-10-12,57561.0,57468.0,57633.0,56443.0,6.46K,0.16%
1,2021-10-11,57468.0,54701.0,57802.5,54430.0,7.51K,5.06%
2,2021-10-10,54701.0,54978.0,56414.0,54173.0,5.84K,-0.50%
3,2021-10-09,54978.0,53925.0,55443.7,53711.0,2.44K,1.95%


In [ ]:
#Menampilkan informasi data frame yang sebenarnya
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3537 entries, 0 to 3536
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      3537 non-null   datetime64[ns]
 1   Price     3537 non-null   float64       
 2   Open      3537 non-null   float64       
 3   High      3537 non-null   float64       
 4   Low       3537 non-null   float64       
 5   Vol.      3537 non-null   object        
 6   Change %  3537 non-null   object        
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 221.1+ KB


In [ ]:
#Menampilkan deskripsi data frame sebenarnya
df.describe()

,Price,Open,High,Low
count,3537.000000,3537.000000,3537.000000,3537.000000
mean,6882.912299,6867.310913,7060.137235,6666.455160
std,12352.348156,12323.964251,12664.826666,11974.408705
min,4.200000,4.200000,4.200000,4.200000
25%,273.300000,273.300000,279.600000,267.300000
50%,872.600000,871.400000,901.500000,833.700000
75%,8230.100000,8230.000000,8423.500000,8019.700000
max,63518.000000,63518.000000,64374.000000,62348.000000


In [ ]:
#Membuat variabel berdasarkan kelompok tanggal
avgp = df.groupby('Date')['Price'].mean()

In [ ]:
#Memecah variabel untuk menentukan index dan values yang digunakan
x = avgp.index
y = avgp.values
print(x,"\n" ,y)

(3537, 3) 
 (3537,)


In [ ]:
#Membuat grafik pergerakan harga Bitcoin
plt.figure(figsize=(16,8))
plt.scatter(x,y)
plt.plot(x,y)
plt.xlabel('Tahun')
plt.ylabel('Pergerakan Harga')
plt.title('Pergerakan Harga Bitcoin')

In [ ]:
#Mencari prediksi rata-rata harga tahun 2021 menggunakan regresi linier
linreg = LinearRegression()
x=np.array(x).reshape(-1,1)
linreg.fit(x,y)

LinearRegression()

In [ ]:
P_2021 = np.array(2021).reshape(-1,1)
pred_P = linreg.predict(P_2021)

In [ ]:
print('Prediksi rata-rata harga tahun 2021 = ', pred_P.item())

Prediksi rata-rata harga tahun 2021 =  56676.66373788958


In [ ]:
#Mencari ikhtisar regresi berganda dari pengaruh Harga tinggi dan Harga buka terhadap Harga
regB = statsmodels.formula.api.ols('Price~High+Open', data = df).fit()
print(regB.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 1.897e+06
Date:                Fri, 29 Oct 2021   Prob (F-statistic):               0.00
Time:                        07:19:30   Log-Likelihood:                -25999.
No. Observations:                3537   AIC:                         5.200e+04
Df Residuals:                    3534   BIC:                         5.202e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4524      7.257      0.062      0.9

In [ ]:
#Mencari ikhtisar regresi berganda dari pengaruh Harga tinggi dan Harga Rendah terhadap Harga
regB = statsmodels.formula.api.ols('Price~High+Low', data = df).fit()
print(regB.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 3.060e+06
Date:                Fri, 29 Oct 2021   Prob (F-statistic):               0.00
Time:                        07:19:38   Log-Likelihood:                -24437.
No. Observations:                3537   AIC:                         4.888e+04
Df Residuals:                    3533   BIC:                         4.891e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.8319      4.666      0.607      0.5

In [ ]:
#Mencari ikhtisar regresi berganda dari pengaruh Harga rendah dan Harga Buka terhadap Harga
regB = statsmodels.formula.api.ols('Price~Low+Open', data = df).fit()
print(regB.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 1.538e+06
Date:                Thu, 28 Oct 2021   Prob (F-statistic):               0.00
Time:                        18:39:28   Log-Likelihood:                -26369.
No. Observations:                3537   AIC:                         5.274e+04
Df Residuals:                    3534   BIC:                         5.276e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      8.9236      8.056      1.108      0.2

In [ ]:
#Mencari ikhtisar regresi berganda dari pengaruh Harga tinggi dan Harga terhadap Harga Buka
regB = statsmodels.formula.api.ols('Open~High+Price', data = df).fit()
print(regB.summary())

                            OLS Regression Results                            
Dep. Variable:                   Open   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 1.913e+06
Date:                Thu, 28 Oct 2021   Prob (F-statistic):               0.00
Time:                        18:39:28   Log-Likelihood:                -25976.
No. Observations:                3537   AIC:                         5.196e+04
Df Residuals:                    3534   BIC:                         5.198e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.5957      7.210      0.083      0.9

In [ ]:
#Mencari ikhtisar regresi berganda dari pengaruh Harga Rendah dan Harga terhadap Harga Buka
regB = statsmodels.formula.api.ols('Open~Low+Price', data = df).fit()
print(regB.summary())

                            OLS Regression Results                            
Dep. Variable:                   Open   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 1.185e+06
Date:                Thu, 28 Oct 2021   Prob (F-statistic):               0.00
Time:                        18:39:29   Log-Likelihood:                -26822.
No. Observations:                3537   AIC:                         5.365e+04
Df Residuals:                    3534   BIC:                         5.367e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     10.1511      9.158      1.108      0.2